In [222]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as p
#np.experimental_enable_numpy_behavior()
#tf.config.list_physical_devices('GPU')

In [223]:
train_df = pd.read_csv('train.csv')
split_index = train_df.iloc[-1].Id
test_df = pd.read_csv('test.csv')
print(len(test_df))
df = pd.concat((train_df, test_df), sort=False)
df.head()

1459


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


In [224]:
for column in df.columns:
  nnull = df[column].isnull().sum()
  
  if nnull > 0:
    print(column + ':', nnull, '/', len(df[column]), 'Null')

MSZoning: 4 / 2919 Null
LotFrontage: 486 / 2919 Null
Alley: 2721 / 2919 Null
Utilities: 2 / 2919 Null
Exterior1st: 1 / 2919 Null
Exterior2nd: 1 / 2919 Null
MasVnrType: 24 / 2919 Null
MasVnrArea: 23 / 2919 Null
BsmtQual: 81 / 2919 Null
BsmtCond: 82 / 2919 Null
BsmtExposure: 82 / 2919 Null
BsmtFinType1: 79 / 2919 Null
BsmtFinSF1: 1 / 2919 Null
BsmtFinType2: 80 / 2919 Null
BsmtFinSF2: 1 / 2919 Null
BsmtUnfSF: 1 / 2919 Null
TotalBsmtSF: 1 / 2919 Null
Electrical: 1 / 2919 Null
BsmtFullBath: 2 / 2919 Null
BsmtHalfBath: 2 / 2919 Null
KitchenQual: 1 / 2919 Null
Functional: 2 / 2919 Null
FireplaceQu: 1420 / 2919 Null
GarageType: 157 / 2919 Null
GarageYrBlt: 159 / 2919 Null
GarageFinish: 159 / 2919 Null
GarageCars: 1 / 2919 Null
GarageArea: 1 / 2919 Null
GarageQual: 159 / 2919 Null
GarageCond: 159 / 2919 Null
PoolQC: 2909 / 2919 Null
Fence: 2348 / 2919 Null
MiscFeature: 2814 / 2919 Null
SaleType: 1 / 2919 Null
SalePrice: 1459 / 2919 Null


In [225]:
# Replace NaN values with 'NA' in columns where NaN is a category
df.Alley.fillna('NA', inplace=True)
df.BsmtQual.fillna('NA', inplace=True)
df.BsmtCond.fillna('NA', inplace=True)
df.BsmtExposure.fillna('NA', inplace=True)
df.BsmtFinType1.fillna('NA', inplace=True)
df.BsmtFinType2.fillna('NA', inplace=True)
df.FireplaceQu.fillna('NA', inplace=True)
df.GarageType.fillna('NA', inplace=True)
df.GarageFinish.fillna('NA', inplace=True)
df.GarageQual.fillna('NA', inplace=True)
df.GarageCond.fillna('NA', inplace=True)
df.PoolQC.fillna('NA', inplace=True)
df.Fence.fillna('NA', inplace=True)
df.MiscFeature.fillna('NA', inplace=True)

'''df.MSZoning.fillna('A', inplace=True)
df.LotFrontage.fillna(df.LotFrontage.mean(), inplace=True)
df.Utilities.fillna('AllPub', inplace=True)
df.Exterior1st.fillna('Other', inplace=True)
df.Exterior2nd.fillna('Other', inplace=True)
df.MasVnrType.fillna('BrkCmn', inplace=True)
df.MasVnrArea.fillna(df.MasVnrArea.mean(), inplace=True)
df.BsmtFinSF1.fillna(df.BsmtFinSF1.mean(), inplace=True)'''

for column in df.columns:
  if df[column].dtype == 'object':
    df[column] = df[column].fillna(df[column].mode(dropna=True).values[0])
  else:
    df[column] = df[column].fillna(df[column].mean())

for column in df.columns:
  nnull = df[column].isna().sum()
  
  if nnull > 0:
    print(column + ':', nnull, '/', len(df[column]), 'Null')

In [226]:
df.LotFrontage.fillna(df.LotFrontage.mean(), inplace=True)
df.GarageYrBlt.fillna(df.GarageYrBlt.mode(), inplace=True)
df.SalePrice.fillna(0, inplace=True)
#df.dropna(axis='rows', inplace=True)

for column in df.columns:
  nnull = df[column].isnull().sum()
  
  if nnull > 0:
    print(column + ':', nnull, '/', len(df[column]), 'Null')

# Should be no NaNs past this point

In [227]:
# Define a function to help with converting columns to one-hot categorical

def col_to_categorical(df, col):
  categorical = pd.get_dummies(df[col])

  for value in categorical.columns:
    df[col + '_' + value] = categorical[value]
  
  df.drop(col, axis='columns', inplace=True)

# Define a convenience function for converting multiple columns to one-hot categorical

def cols_to_categorical(df, cols):
  for col in cols:
    col_to_categorical(df, col)

In [228]:
categorical_cols = [
  'MSZoning',
  'Street',
  'Alley',
  'LotShape',
  'LandContour',
  'Utilities',
  'LotConfig',
  'LandSlope',
  'Neighborhood',
  'Condition1',
  'Condition2',
  'BldgType',
  'HouseStyle',
  'RoofStyle',
  'RoofMatl',
  'Exterior1st',
  'Exterior2nd',
  'MasVnrType',
  'ExterQual',
  'ExterCond',
  'Foundation',
  'BsmtQual',
  'BsmtCond',
  'BsmtExposure',
  'BsmtFinType1',
  'BsmtFinType2',
  'Heating',
  'HeatingQC',
  'CentralAir',
  'Electrical',
  'KitchenQual',
  'Functional',
  'FireplaceQu',
  'GarageType',
  'GarageFinish',
  'GarageQual',
  'GarageCond',
  'PavedDrive',
  'PoolQC',
  'Fence',
  'MiscFeature',
  'SaleType',
  'SaleCondition'
]

In [229]:
cols_to_categorical(df, categorical_cols)
df.columns

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       ...
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=304)

In [230]:
df.set_index('Id', drop=True, inplace=True)

In [231]:
from sklearn.preprocessing import minmax_scale

# Scale all columns
for column in df.columns:
  if column != 'SalePrice':
    df[column] = pd.Series(minmax_scale(df[column]), index=df.index)

In [232]:
from sklearn.preprocessing import MinMaxScaler

target_col = df.loc[:split_index].SalePrice
target_scaler = MinMaxScaler()
Y = np.array(target_col.to_numpy())
Y = Y.reshape(Y.size, 1)
target_scaler.fit(Y)
Y = target_scaler.transform(Y)
Y = Y.reshape(Y.size)
df.drop('SalePrice', axis='columns', inplace=True)

In [233]:
train_df = df.loc[:split_index]

for i in range(split_index + 1, split_index + 100):
  if i in df.index:
    test_df = df.loc[i:]
    break

train_df.to_csv('train_out.csv')
test_df.to_csv('test_out.csv')
train_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,0.235294,0.150685,0.033420,0.666667,0.500,0.949275,0.883333,0.12250,0.125089,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.000000,0.202055,0.038795,0.555556,0.875,0.753623,0.433333,0.00000,0.173281,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.235294,0.160959,0.046507,0.666667,0.500,0.934783,0.866667,0.10125,0.086109,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.294118,0.133562,0.038561,0.666667,0.500,0.311594,0.333333,0.00000,0.038271,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.235294,0.215753,0.060576,0.777778,0.500,0.927536,0.833333,0.21875,0.116052,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [234]:
print(train_df.shape, test_df.shape, target_col.shape)

(1460, 302) (1459, 302) (1460,)


In [235]:
X_train = np.array(train_df.to_numpy())
print(X_train.shape, Y.shape)

(1460, 302) (1460,)


In [236]:
train_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,0.235294,0.150685,0.033420,0.666667,0.500,0.949275,0.883333,0.12250,0.125089,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.000000,0.202055,0.038795,0.555556,0.875,0.753623,0.433333,0.00000,0.173281,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.235294,0.160959,0.046507,0.666667,0.500,0.934783,0.866667,0.10125,0.086109,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.294118,0.133562,0.038561,0.666667,0.500,0.311594,0.333333,0.00000,0.038271,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
5,0.235294,0.215753,0.060576,0.777778,0.500,0.927536,0.833333,0.21875,0.116052,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [237]:
def mse(X, Y):
  return ((X - Y) ** 2).mean() / 2

In [238]:
def linear_regression(x, w, b):
  return np.dot(x, w) + b

print(linear_regression([2], [3], 2))

8


In [239]:
def gradient_descent(X, Y, w, _b, learning_rate, iterations):
  b = _b
  double_len_X = len(X) * 2
  X_transpose = X.transpose()

  for _ in range(iterations):
    #linear_regressions = np.array([linear_regression(x, w, b) for x in X])
    linear_regressions = linear_regression(X, w, b)
    differences = (linear_regressions - Y)

    '''for i in range(len(w)):
      w_change = (differences * X[:, i]).mean()

      #for j in range(len(X)):
      #  w_change += differences[j] * X[j][i]
      
      w[i] -= learning_rate * w_change / 2'''
    
    w -= learning_rate * np.dot(X_transpose, differences) / double_len_X
    b_change = differences.mean()

    '''for j in range(len(X)):
      b_change += (linear_regressions[j] - Y[j])'''
    
    b -= learning_rate * b_change / 2

    if _ % 10000 == 0:
      print('Iteration: ' + str(_) + ', Loss: ' + str(mse(linear_regressions, Y)))
  
  return b

In [240]:
w = np.array([0] * 302, dtype='float')
b = gradient_descent(X_train, Y, w, 0, 0.01, 100000)

Iteration: 0, Loss: 0.026640919287270397
Iteration: 10000, Loss: 0.0007303041545136871
Iteration: 20000, Loss: 0.0006544633281726532
Iteration: 30000, Loss: 0.0006205399799965043
Iteration: 40000, Loss: 0.0005986382461302093
Iteration: 50000, Loss: 0.0005823097158760736
Iteration: 60000, Loss: 0.0005692169563439769
Iteration: 70000, Loss: 0.0005582630400169426
Iteration: 80000, Loss: 0.0005488493856648091
Iteration: 90000, Loss: 0.0005406124155676453


In [241]:
print(list(w), b)

[-0.020616425613333184, -0.019436495560088342, 0.057238817465220374, 0.09682168571950414, 0.050971289256004124, 0.01442395309866415, 0.006622463185966146, 0.037947344103421494, 0.03876450559473981, 0.015517155383109478, 0.009188992318057929, 0.0431966505313144, 0.0860348262540955, 0.09188489007182257, -0.0034281926414083948, 0.11222805365930799, 0.025574454911627322, 0.0017408500294780133, 0.052399886208865244, 0.011502485088038354, -0.013627118596858629, -0.029433124887636823, 0.0639650513776584, 0.03680980011816021, -0.013190384490354786, 0.06237405139345157, 0.02288841316014696, 0.038888348667432925, 0.0038352375947737446, 0.001764464535079337, 0.024997457363060048, 0.028724685774413755, 0.026195754621889236, -0.0016449856727396297, -0.005863979712146897, -0.003858692956208466, -0.03503750210878714, 0.014268869658591303, 0.004876111171303101, 0.005813056003488412, 0.0027160461566798693, -0.013155785742386229, 0.0057923666236608965, -0.0030427045402522408, -0.004726204056587326, 0.00

In [242]:
def sse(X, Y):
  return ((X - Y) ** 2).sum()

In [243]:
model_sse = sse(linear_regression(X_train, w, b), Y)
sst = sse(np.ones(len(Y)) * Y.mean(), Y)
print(1 - model_sse / sst)

0.9123021843622661


In [244]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, Y)
model.score(X_train, Y)

0.9332152454506968

In [245]:
sklearn_predictions = model.predict(test_df.to_numpy())

In [246]:
predictions = linear_regression(test_df.to_numpy(), w, b)

In [248]:
predictions = predictions.reshape((predictions.size, 1))
predictions = target_scaler.inverse_transform(predictions)
predictions = predictions.reshape(predictions.size)
predictions

array([108403.69764999, 146039.1664962 , 173878.95959025, ...,
       163117.27916203, 101996.17643232, 219999.73304765])